In [13]:
import pretty_midi
from note import Note
import processor
import json
from data import Data, preprocess_midi_files_under
import torch
from model import load_config, BigramLanguageModel, load_model

In [14]:
preprocess_midi_files_under('C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2018', 'preprocess')

 [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2018\AbdelmoulaJS01.MID] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2018\AbdelmoulaJS02.MID] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2018\AbdelmoulaJS03.MID] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2018\AbdelmoulaJS04.MID] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2018\AbdelmoulaJS05.MID] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2018\AbdelmoulaJS06.MID] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2018\AbdelmoulaJS07.MID] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competition/2018\AbdelmoulaJS08.MID] [C:/Users/Draco/Documents/GitHub/MusicTransformer-pytorch/dataset/midi_piano_competitio

In [15]:
# Example usage:
config = load_config()
batch_size = config["batch_size"]
block_size = config["block_size"]
max_iters = config["max_iters"]
eval_interval = config["eval_interval"]
save_interval = config["save_interval"]
learning_rate = config["learning_rate"]
eval_iters = config["eval_iters"]
n_embd = config["n_embd"]
n_head = config["n_head"]
n_layer = config["n_layer"]
dropout = config["dropout"]
vocab_size = config["vocab_size"]
device = config["device"]
generated_event_length = config["generated_event_length"]

In [ ]:
dataset = Data('preprocess').all_data()

torch.manual_seed(1337)

m = load_model()

data = torch.tensor(Data('preprocess').all_data())
# data = torch.tensor(dataset.batch(1,2000)[2], dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = m(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


0.908165 M parameters
step 0: train loss 6.1152, val loss 6.1125
step 100: train loss 3.2869, val loss 3.3482
step 200: train loss 3.1741, val loss 3.2404
step 300: train loss 3.1268, val loss 3.1854


In [25]:
torch.save(m.state_dict(), f'models/train_{losses["train"]:.4f}-val_{losses["val"]:.4f}.pth')
# m = load_model('2000_epochs.pth')

In [11]:
# generate from the model
context = torch.tensor([[306, 59, 150]], device=device)
output_sequence = m.generate(context, max_new_tokens=200)[0].tolist()

In [12]:
processor.decode_midi(output_sequence, "thing.mid")

In [ ]:
processor.encode_fl_notes